<a href="https://colab.research.google.com/github/RostislavStoyanov/fmi-irkd-kdd-2021/blob/main/cudf_GPU_dataframes_RostislavStoyanov26321/01_Intro_to_cuDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intro to cuDF
=======================

Welcome to first cuDF tutorial notebook! This is a short introduction to cuDF, partly modeled after 10 Minutes to Pandas, geared primarily for new users. cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data. While we'll only cover some of the cuDF functionality, but at the end of this tutorial we hope you'll feel confident creating and analyzing GPU DataFrames.

We'll start by introducing the pandas library, and quickly move on cuDF.

In [ ]:
import os
import numpy as np
import math
np.random.seed(12)

# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ["CONDA_PREFIX"] = "/usr/local"
for so in ['cudf']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
# fix for BlazingSQL import issue
# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by /usr/local/lib/python3.7/site-packages/../../libblazingsql-engine.so)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.7/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.7/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 14.78 MiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
	os.environ['CONDA_PREFIX'] = '/usr/local'
	for so in ['cudf', 'rmm',

<a id="pandas"></a>
## Pandas

Data scientists typically work with two types of data: unstructured and structured. Unstructured data often comes in the form of text, images, or videos. Structured data - as the name suggests - comes in a structured form, often represented by a table or CSV. We'll focus the majority of these tutorials on working with structured data.

There exist many tools in the Python ecosystem for working with structured, tabular data but few are as widely used as Pandas. Pandas represents data in a table and allows a data scientist to manipulate the data to perform a number of useful operations such as filtering, transforming, aggregating, merging, visualizing and many more. 

For more information on Pandas, check out the excellent documentation: http://pandas.pydata.org/pandas-docs/stable/

Below we show how to create a Pandas DataFrame, an internal object for representing tabular data.

In [ ]:
import pandas as pd; print('Pandas Version:', pd.__version__)


# here we create a Pandas DataFrame with
# two columns named "key" and "value"
df = pd.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

Pandas Version: 1.1.5
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


We can perform many operations on this data. For example, let's say we wanted to sum all values in the in the `value` column. We could accomplish this using the following syntax:

In [ ]:
aggregation = df['value'].sum()
print(aggregation)

60.0


<a id="cudf"></a>
## cuDF

Pandas is fantastic for working with small datasets that fit into your system's memory and workflows that are not computationally intense. However, datasets are growing larger and data scientists are working with increasingly complex workloads - the need for accelerated computing is increasing rapidly.

cuDF is a package within the RAPIDS ecosystem that allows data scientists to easily migrate their existing Pandas workflows from CPU to GPU, where computations can leverage the immense parallelization that GPUs provide.

Below, we show how to create a cuDF DataFrame.

In [ ]:
import cudf; print('cuDF Version:', cudf.__version__)


# here we create a cuDF DataFrame with
# two columns named "key" and "value"
df = cudf.DataFrame()
df['key'] = [0, 0, 2, 2, 3]
df['value'] = [float(i + 10) for i in range(5)]
print(df)

cuDF Version: 0.19.2
   key  value
0    0   10.0
1    0   11.0
2    2   12.0
3    2   13.0
4    3   14.0


As before, we can take this cuDF DataFrame and perform a `sum` operation over the `value` column. The key difference is that any operations we perform using cuDF use the GPU instead of the CPU.

In [ ]:
aggregation = df['value'].sum()
print(aggregation)

60.0


Note how the syntax for both creating and manipulating a cuDF DataFrame is identical to the syntax necessary to create and manipulate Pandas DataFrames; the cuDF API is based on the Pandas API. This design choice minimizes the cognitive burden of switching from a CPU based workflow to a GPU based workflow and allows data scientists to focus on solving problems while benefitting from the speed of a GPU!

# DataFrame Basics with cuDF

In the following tutorial, you'll get a chance to familiarize yourself with cuDF. For those of you with experience using pandas, this should look nearly identical.

Along the way you'll notice small exercises. These exercises are designed to help you get a feel for writing the code yourself, but if you get stuck, you can take a look at the solutions.

Portions of this were borrowed from the 10 Minutes to cuDF guide.

Object Creation
---------------

Creating a `cudf.Series`.

In [ ]:
s = cudf.Series([1,2,3,None,4])
print(s)

0       1
1       2
2       3
3    <NA>
4       4
dtype: int64


Creating a `cudf.DataFrame` by specifying values for each column.

In [ ]:
df = cudf.DataFrame({'a': list(range(20)),
'b': list(reversed(range(20))),
'c': list(range(20))})
print(df)

     a   b   c
0    0  19   0
1    1  18   1
2    2  17   2
3    3  16   3
4    4  15   4
5    5  14   5
6    6  13   6
7    7  12   7
8    8  11   8
9    9  10   9
10  10   9  10
11  11   8  11
12  12   7  12
13  13   6  13
14  14   5  14
15  15   4  15
16  16   3  16
17  17   2  17
18  18   1  18
19  19   0  19


Creating a `cudf.DataFrame` from a `pd.Dataframe`.

In [ ]:
pdf = pd.DataFrame({'a': [0, 1, 2, 3],'b': [0.1, 0.2, None, 0.3]})
gdf = cudf.DataFrame.from_pandas(pdf)
print(gdf)

   a     b
0  0   0.1
1  1   0.2
2  2  <NA>
3  3   0.3


Viewing Data
-------------

Viewing the top rows of a GPU dataframe.

In [ ]:
print(df.head(2))

   a   b  c
0  0  19  0
1  1  18  1


Sorting by values.

In [ ]:
print(df.sort_values('b'))

     a   b   c
19  19   0  19
18  18   1  18
17  17   2  17
16  16   3  16
15  15   4  15
14  14   5  14
13  13   6  13
12  12   7  12
11  11   8  11
10  10   9  10
9    9  10   9
8    8  11   8
7    7  12   7
6    6  13   6
5    5  14   5
4    4  15   4
3    3  16   3
2    2  17   2
1    1  18   1
0    0  19   0


Selection
------------

## Getting

Selecting a single column, which initially yields a `cudf.Series` (equivalent to `df.a`).

In [ ]:
print(df['a'])

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
Name: a, dtype: int64


## Selection by Label

Selecting rows from index 2 to index 5 from columns `a` and `b`.

In [ ]:
print(df.loc[2:5, ['a', 'b']])

   a   b
2  2  17
3  3  16
4  4  15
5  5  14


## Selection by Position

Selecting via integers and integer slices, like numpy/pandas.

In [ ]:
print(df.iloc[0])

/usr/local/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/usr/local/lib/python3.7/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice/.

For more information about alternatives visit: ('https://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))


a     0
b    19
c     0
Name: 0, dtype: int64


In [ ]:
print(df.iloc[0:3, 0:2])

   a   b
0  0  19
1  1  18
2  2  17


You can also select elements of a `DataFrame` or `Series` with direct index access.

In [ ]:
print(df[3:5])

   a   b  c
3  3  16  3
4  4  15  4


In [ ]:
print(s[3:5])

3    <NA>
4       4
dtype: int64


## Exercise 1

Try to select only the rows at index `4` and `9` from `df`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.iloc[[4,9]])
   </pre>
</details>

In [ ]:
print(df.iloc[[4,9]])

   a   b  c
4  4  15  4
9  9  10  9


## Boolean Indexing

Selecting rows in a `DataFrame` or `Series` by direct Boolean indexing.

In [ ]:
print(df[df.b > 15])

   a   b  c
0  0  19  0
1  1  18  1
2  2  17  2
3  3  16  3


Selecting values from a `DataFrame` where a Boolean condition is met, via the `query` API.

In [ ]:
print(df.query("b == 3"))  

     a  b   c
16  16  3  16


In [ ]:
val = 3
df.query("b == @val")

,a,b,c
16,16,3,16


You can also pass local variables to cuDF queries, via the `local_dict` keyword or `@` operator.

In [ ]:
cudf_comparator = 3
print(df.query("b == @cudf_comparator"))

     a  b   c
16  16  3  16


Supported logical operators include `>`, `<`, `>=`, `<=`, `==`, and `!=`.

## Exercise 2

Try to select only the rows from `df` where the value in column `b` is greater than the value in column `c` + 6.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.query("b > c + 6"))
   </pre>
</details>

In [ ]:
print(df.query("b > c + 6"))

   a   b  c
0  0  19  0
1  1  18  1
2  2  17  2
3  3  16  3
4  4  15  4
5  5  14  5
6  6  13  6


Missing Data
------------

Missing data can be replaced by using the `fillna` method.

In [ ]:
print(s.fillna(999))

0      1
1      2
2      3
3    999
4      4
dtype: int64


Operations
------------

## Stats

Calculating descriptive statistics for a `Series`.

In [ ]:
s = cudf.Series(np.arange(10)).astype(np.float32)

In [ ]:
print(s.mean(), s.var())

4.5 9.166666666666668


In [ ]:
print(df.describe())

              a         b         c
count  20.00000  20.00000  20.00000
mean    9.50000   9.50000   9.50000
std     5.91608   5.91608   5.91608
min     0.00000   0.00000   0.00000
25%     4.75000   4.75000   4.75000
50%     9.50000   9.50000   9.50000
75%    14.25000  14.25000  14.25000
max    19.00000  19.00000  19.00000


## Applymap

Applying functions to a `Series`.

In [ ]:
def add_ten(num):
    return num + 10

print(s.applymap(add_ten))

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64


We can also apply more complicated functions, such as this one.

In [ ]:
def complex_math_transform(num):
    return math.cos(num) * 3 / 9

def not_so_complex_transform(num):
    return num * 3 / 9

#print(s.applymap(complex_math_transform))
print(s.cos().applymap(not_so_complex_transform)) # not actually correct

0    0.333333
1    0.180101
2   -0.138716
3   -0.329997
4   -0.217881
5    0.094554
6    0.320057
7    0.251301
8   -0.048500
9   -0.303710
dtype: float64


## String Methods

Like pandas, cuDF provides string processing methods in the `str` attribute of `Series`. Full documentation of string methods is a work in progress. Please see the cuDF and [nvStrings API](https://docs.rapids.ai/api/nvstrings/nightly/) documentation for more information.

In [ ]:
s = cudf.Series(['A', 'B', 'C', 'Aaba', 'Baca', None, 'CABA', 'dog', 'cat'])
print(s.str.lower())

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: object


## Exercise 3

Try to convert all the strings to uppercase. Take a look at the nvStrings API documentation linked above if you need some help.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(s.str.upper())
   </pre>
</details>

In [ ]:
print(s.str.upper())

0       A
1       B
2       C
3    AABA
4    BACA
5    <NA>
6    CABA
7     DOG
8     CAT
dtype: object


## Concat

Concatenating `Series` and `DataFrames` row-wise.

In [ ]:
s = cudf.Series([1, 2, 3, None, 5])
print(cudf.concat([s, s]))
print("---\nAlso concat by cols:")
print(cudf.concat([s,s], axis=1))

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64
---
Also concat by cols:
      0     1
0     1     1
1     2     2
2     3     3
3  <NA>  <NA>
4     5     5


## Append

Appending values from another `Series` or array-like object.

In [ ]:
print(s.append(s))

0       1
1       2
2       3
3    <NA>
4       5
0       1
1       2
2       3
3    <NA>
4       5
dtype: int64


## Join

Performing SQL style merges. Note that the dataframe order is not maintained, but may be restored post-merge by sorting by the index.

In [ ]:
df_a = cudf.DataFrame()
df_a['key'] = ['a', 'b', 'c', 'd', 'e']
df_a['vals_a'] = [float(i + 10) for i in range(5)]

df_b = cudf.DataFrame()
df_b['key'] = ['a', 'c', 'e']
df_b['vals_b'] = [float(i+100) for i in range(3)]

merged = df_a.merge(df_b, on=['key'], how='left')
print(merged)

  key  vals_a vals_b
0   a    10.0  100.0
1   c    12.0  101.0
2   e    14.0  102.0
3   b    11.0   <NA>
4   d    13.0   <NA>


## Exercise 4

Using the DataFrames we created above, try to do an `inner` join using `merge`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df_a.merge(df_b, on=['key'], how='inner'))
   </pre>
</details>

In [ ]:
inner_merge = df_a.merge(df_b, on=['key'])
print(inner_merge)

  key  vals_a  vals_b
0   a    10.0   100.0
1   c    12.0   101.0
2   e    14.0   102.0


## Grouping

Like pandas, cuDF supports the [Split-Apply-Combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) groupby paradigm.

In [ ]:
df['agg_col1'] = [1 if x % 2 == 0 else 0 for x in range(len(df))]
df['agg_col2'] = [1 if x % 3 == 0 else 0 for x in range(len(df))]

Grouping and then applying the `sum` function to the grouped data.

In [ ]:
print(df.groupby('agg_col1').sum())

            a    b    c  agg_col2
agg_col1                         
1          90  100   90         4
0         100   90  100         3


Grouping and applying statistical functions to specific columns, using `agg`.

In [ ]:
print(df.groupby('agg_col1').agg({'a':'max', 'b':'mean', 'c':'sum'}))

           a     b    c
agg_col1               
1         18  10.0   90
0         19   9.0  100


## Exercise 5

We can also group by multiple columns at once, which we call grouping hierarchically. Try to group `df` by `agg_col1` and `agg_col2` and calculate the mean of column `a` and minimum of column `b`.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>print(df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'}).to_pandas())
   </pre>
</details>

In [ ]:
print(df.groupby(['agg_col1', 'agg_col2']).agg({'a':'mean', 'b':'min'}))

                           a  b
agg_col1 agg_col2              
1        0          9.000000  3
0        0         10.428571  0
1        1          9.000000  1
0        1          9.000000  4


Time Series
------------


`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [ ]:
date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('11/20/2018', periods=72, freq='D')
date_df['value'] = np.random.sample(len(date_df))
print(date_df.head())

        date     value
0 2018-11-20  0.154163
1 2018-11-21  0.740050
2 2018-11-22  0.263315
3 2018-11-23  0.533739
4 2018-11-24  0.014575


## Exercise 6

Try to use `query` to filter `date_df` to only those row with a date before `2018-11-23`. This is a bit trickier than the prior exercises. As a hint, you'll want to explore the `to_datetime` function from the `pandas` library.

<details><summary><b>Solution</b></summary>
   <pre>
    <br>
    search_date = pd.to_datetime('2018-11-23')
    print(date_df.query("date &lt= @search_date"))
            </br>
   </pre>
</details>

In [ ]:
filter_datetime = pd.to_datetime('2018-11-23')
print(date_df.query("date < @filter_datetime"))

        date     value
0 2018-11-20  0.154163
1 2018-11-21  0.740050
2 2018-11-22  0.263315


You can also interact with datetime columns to extract things like the day, hour, minute, and more.

In [ ]:
date_df['minute'] = date_df.date.dt.minute
print(date_df.head())

        date     value  minute
0 2018-11-20  0.154163       0
1 2018-11-21  0.740050       0
2 2018-11-22  0.263315       0
3 2018-11-23  0.533739       0
4 2018-11-24  0.014575       0


Converting Data Representation
--------------------------------

## Pandas

Converting a cuDF `DataFrame` to a pandas `DataFrame`.

In [ ]:
print(df.head().to_pandas())

   a   b  c  agg_col1  agg_col2
0  0  19  0         1         1
1  1  18  1         0         0
2  2  17  2         1         0
3  3  16  3         0         1
4  4  15  4         1         0


## Numpy

Converting a cuDF `DataFrame` to a numpy `ndarray`.

In [ ]:
print(df.as_matrix())

[[ 0 19  0  1  1]
 [ 1 18  1  0  0]
 [ 2 17  2  1  0]
 [ 3 16  3  0  1]
 [ 4 15  4  1  0]
 [ 5 14  5  0  0]
 [ 6 13  6  1  1]
 [ 7 12  7  0  0]
 [ 8 11  8  1  0]
 [ 9 10  9  0  1]
 [10  9 10  1  0]
 [11  8 11  0  0]
 [12  7 12  1  1]
 [13  6 13  0  0]
 [14  5 14  1  0]
 [15  4 15  0  1]
 [16  3 16  1  0]
 [17  2 17  0  0]
 [18  1 18  1  1]
 [19  0 19  0  0]]


Converting a cuDF `Series` to a numpy `ndarray`.

In [ ]:
print(df['a'].to_array())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Getting Data In/Out
------------------------


## CSV

Writing to a CSV file, by first sending data to a pandas `Dataframe` on the host.

In [ ]:
if not os.path.exists('example_output'):
    os.mkdir('example_output')
    
df.to_pandas().to_csv('example_output/foo.csv', index=False)

Reading from a csv file.

In [ ]:
df = cudf.read_csv('example_output/foo.csv')
print(df)

     a   b   c  agg_col1  agg_col2
0    0  19   0         1         1
1    1  18   1         0         0
2    2  17   2         1         0
3    3  16   3         0         1
4    4  15   4         1         0
5    5  14   5         0         0
6    6  13   6         1         1
7    7  12   7         0         0
8    8  11   8         1         0
9    9  10   9         0         1
10  10   9  10         1         0
11  11   8  11         0         0
12  12   7  12         1         1
13  13   6  13         0         0
14  14   5  14         1         0
15  15   4  15         0         1
16  16   3  16         1         0
17  17   2  17         0         0
18  18   1  18         1         1
19  19   0  19         0         0


That's it! You've got the basics of cuDF down! Let's talk a little bit about the computational performance of cuDF and GPUs.

# Performace

One of the primary reasons to use cuDF over pandas is performance. For some workflows, the GPU can be **much** faster than the CPU. Let's illustrate this by starting with a small example: creating a DataFrame and calculating the sum of a column.

In [ ]:
a = np.random.rand(10000000) # 10 million values

In [ ]:
pdf = pd.DataFrame()
cdf = cudf.DataFrame()

In [ ]:
%time pdf['a'] = a
%time cdf['a'] = a

CPU times: user 617 ms, sys: 417 ms, total: 1.03 s
Wall time: 967 ms
CPU times: user 21.3 ms, sys: 63.7 ms, total: 85 ms
Wall time: 84.1 ms


In [ ]:
%%timeit
pdf['a'].sum()

10 loops, best of 5: 41.4 ms per loop


In [ ]:
%%timeit
cdf['a'].sum()

1000 loops, best of 5: 1.07 ms per loop


Pretty cool! This is a pretty small example, though.

### A More Realistic Example: Sensor Data Analytics

To get a more realistic sense of how powerful cuDF and GPUs can be, let's imagine you had a fleet of sensors that collect data every millisecond. These sensors could be measuring pressure, temperature, or something else entirely.

Let's imagine we want to analyze one day's worth of sensor data. We'll assign random values for the sensor `value` to use for this example. We'll start by creating the data, and generating some datetime related features like we learned about in the above tutorial.

In [ ]:
%%time

date_df = pd.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute

print(date_df.shape)
date_df.head()

(86400001, 4)
CPU times: user 20.4 s, sys: 10.9 s, total: 31.3 s
Wall time: 31 s


Just creating the data takes a while! Let's do our analysis. From our sensor data, we want to get the maximum sensor value for each minute. Since we don't want to combine values in the same minute of different hours, we'll need to do a hierarchical groupby.

In [ ]:
%time results = date_df.groupby(['hour', 'minute']).agg({'value':'max'})
results.head()

CPU times: user 3.11 s, sys: 8.52 s, total: 11.6 s
Wall time: 11.5 s


value
hour minute          
0    0       0.999994
     1       0.999931
     2       0.999995
     3       0.999962
     4       0.999975

This is fairly slow! Imagine if we had a fleet of sensors. Time would become a serious concern.

Let's try this in cuDF now, using the GPU DataFrame. We'll run the same code as above.

In [ ]:
%%time

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range(start='2019-07-05', end='2019-07-06', freq='ms')
date_df['value'] = np.random.sample(len(date_df))

date_df['hour'] = date_df.date.dt.hour
date_df['minute'] = date_df.date.dt.minute
date_df['second'] = date_df.date.dt.second

print(date_df.shape)
print(date_df.head())

(86400001, 5)
                     date     value  hour  minute  second
0 2019-07-05 00:00:00.000  0.453388     0       0       0
1 2019-07-05 00:00:00.001  0.804337     0       0       0
2 2019-07-05 00:00:00.002  0.449341     0       0       0
3 2019-07-05 00:00:00.003  0.596637     0       0       0
4 2019-07-05 00:00:00.004  0.552345     0       0       0
CPU times: user 1.45 s, sys: 483 ms, total: 1.94 s
Wall time: 1.94 s


In [ ]:
%time results = date_df.groupby(['hour', 'minute', 'second']).agg({'value':'max'})
results.head()

CPU times: user 49.3 ms, sys: 35.2 ms, total: 84.5 ms
Wall time: 84.2 ms


,,,value
hour,minute,second,
2,43,35,0.999544
4,13,8,0.997685
1,44,5,0.999127
3,31,6,0.998985
9,49,13,0.994060


While the results may vary slightly, it should be clear that GPU acceleration can make a significant difference. We can get much faster results with the same code!

## Exercise 7

Play around with some more pandas and cuDF operations and compare the performance between them. What operations can you find that gives the highest performance ratio? 

You can start a cell with `%%time` to time the cell, or with `%%timeit`, which runs the cell multiple times and gives an average. `%%timeit` gives a more accurate benchmark but takes longer to run.